In [1]:
from requests.compat import *
from requests import request
from bs4 import BeautifulSoup
import pandas as pd
import re


def recode_scraper(category1, category2, year):
    url = 'http://statiz.sporki.com/stats'
        
    
    params = {
        'm':'main',
        'm2':category1,
        'm3':category2,
        'year':str(year),
        'pr':'1000',
        'reg':'C30',
        'ls':'1'
    }

    resp = request('GET',url=url, params=params)
    dom = BeautifulSoup(resp.text,'html.parser')
    
    data = [node.text for node in dom.find_all('td')]
    position_data = dom.select('div > span')
    
    players = [item.text for item in dom.select('td > div > a')]
    players_href = [tag.attrs['href'] for tag in dom.select('td> div > a')]
    players_num = [re.search(r'\d+', item).group() for item in players_href]
    attrs_2 = [item['title'] for item in dom.select('li[title]')]
    if (category1 == 'pitching' and category2 == 'pitch'):
        attrs_2[19] = 'Looking Strike Out, 루킹 삼진'
        attrs_2[20] = 'Swing Strike Out, 헛스윙 삼진'
    attrs = attrs_2[:int(len(attrs_2)/2)]
    columns = ['Number', '이름','Position'] + attrs
    df = pd.DataFrame(columns = columns)

    for i in range(len(players)):
        row = {}
        row['Number'] = players_num[i]
        row['이름'] = players[i]
        if (category1 == 'fielding'):
            row['Position'] = position_data[30+3*i].text
        else:
            row['Position'] = position_data[48+3*i].text
        for j in range(len(attrs)):
            index = i*(len(attrs)+4) + 4 + j
            row[attrs[j]] = data[index]

        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    
    return df

In [2]:
def get_position_code(position):
    """각 포지션에 대한 코드를 반환합니다."""
    if position == '1루수':
        return '1B'
    elif position == '2루수':
        return '2B'
    elif position == '3루수':
        return '3B'
    elif position == '유격수':
        return 'SS'
    elif position == '좌익수':
        return 'LF'
    elif position == '중견수':
        return 'CF'
    elif position == '우익수':
        return 'RF'
    elif position == '포수':
        return 'C'
    else:
        return None

def get_raa_sum(records_df, player_number, pos_code):
    try:
        specific_record = records_df[(records_df['Number'] == player_number) & (records_df['Position'] == pos_code)]
        raa = float(specific_record['Fielding Runs Above Average/144 Games, 평균 대비 수비 득점 기여 (144경기 기준)'].iloc[0])
    except (ValueError, IndexError):
        raa_list = list(map(float, records_df[records_df['Number'] == player_number]['Fielding Runs Above Average/144 Games, 평균 대비 수비 득점 기여 (144경기 기준)']))
        raa = sum(raa_list) / len(raa_list) if raa_list else 0
    return raa

def calculate_raa_sums(positions_num, outfield_recode, infield_recode, catcher_recode):
    outfield_raa_sum = infield_raa_sum = catcher_raa_sum = 0.0
    position_keys = list(positions_num.keys())

    for i, player_number in enumerate(positions_num.values()):
        position = position_keys[i]
#         print(position)
        pos_code = get_position_code(position)
#         print(pos_code)
#         print(player_number)
        
        if position in ['우익수', '좌익수', '중견수']:
            outfield_raa_sum += get_raa_sum(outfield_recode, player_number, pos_code)
        elif position in ['1루수', '2루수', '유격수', '3루수']:
            infield_raa_sum += get_raa_sum(infield_recode, player_number, pos_code)
        elif position in ['포수']:
            players_recode = catcher_recode[catcher_recode['Number'] == player_number]
            try:
                raa_include_framing = float(players_recode[players_recode['Position'] == 'C']['Fielding Runs Above Average/144 Games, 평균 대비 수비 득점 기여 (144경기 기준)'])
                raa_framing_144 = (float(players_recode[players_recode['Position'] == 'C']['Fielding Runs (Framing), 프레이밍 관련 득점 기여'])*144) / float(players_recode[players_recode['Position'] == 'C']['Games Played, 출장'])
                raa = raa_include_framing - raa_framing_144
                catcher_raa_sum = catcher_raa_sum + raa
            except:
                raa_list = list(map(float,list(players_recode['Fielding Runs Above Average/144 Games, 평균 대비 수비 득점 기여 (144경기 기준)'])))
                raa = sum(raa_list)/len(raa_list) if raa_list else 0
                catcher_raa_sum = catcher_raa_sum + raa

    return outfield_raa_sum, infield_raa_sum, catcher_raa_sum

In [3]:
from requests.compat import *
from requests import request
from bs4 import BeautifulSoup
import re
import pandas as pd

# 수비 추가 버전

def gamelog_scraper(year):
    
    games_schedule = list()
    
    # 1 ~ 12월까지 경기 ID 수집
    for i in range(1,13):
        url = 'http://statiz.sporki.com/schedule'
    
        params = {
            'year':str(year),
            'month':str(i)
        }
        
        resp = request('GET',url=url, params=params)
        dom = BeautifulSoup(resp.text,'html.parser')
        
        if (['p_schedule'] in list(dom.select('div > ul > li >a')[-1].attrs.values())):
            print('skip')
            continue
        
        print(i)
        games_tag = [tag for tag in dom.select('div > ul > li >a') if 'class' not in list(tag.attrs.keys())]
        games_href = [tag.attrs['href'] for tag in games_tag]
        games_num = [re.search(r'\d+', item).group() for item in games_href]
        
        games_schedule = games_schedule + games_num
    
    # Gamelog feature 지정
    columns = ['날짜', '경기장', 'Pitcher Number','Hitter Number','회', '내야 수비 RAA','외야 수비 RAA','포수 수비 RAA', '투수 이름', 
                '타자 이름', '투구수', '결과', '이전상황','이후상황','LEV','REs','REa','WPe','WPa']
    df = pd.DataFrame(columns = columns)
    
    # 수비 데이터 호출
    outfield_recode = recode_scraper('fielding','outField',2023)
    infield_recode = recode_scraper('fielding','inField',2023)
    catcher_recode = recode_scraper('fielding','catcher',2023)
    
    # 게임 별 Gamelog 수집
    for game_id in games_schedule:
        
        url = 'http://statiz.sporki.com/schedule'
        
        params = {
            'm':'gamelogs',
            's_no':game_id
        }

        resp = request('GET',url=url, params=params)
        dom = BeautifulSoup(resp.text,'html.parser')
        
        stadium = dom.find_all(attrs={'class':'txt'})[1].text[7:9]
        date = dom.find_all(attrs={'class':'txt'})[1].text[11:16]
        print(stadium, date)
        game_canceled = dom.find_all(attrs={'style':'font-size:1.6rem;'})
    
        if (len(game_canceled) > 0):
            continue
        
        data = [node.text for node in dom.find_all('td')]
        lineup_data = data[:80]
        data = data[80:-4]
        
        players = [item.text for item in dom.select('td > a')]
        players_href = [tag.attrs['href'] for tag in dom.select('td > a')]
        players_num = []

        for item in players_href:
            match = re.search(r'\d+', item)
            if match:
                players_num.append(match.group())
            else:
                players_num.append('')
                
        players_num_lineup = players_num[:20]
        lineup_columns = ['Player Number', 'Name', 'Position', 'Bats/Throws', 'Home/Away']
        lineup_df = pd.DataFrame(columns=lineup_columns)

        home_away_toggle = 'Away'  # 초기 팀을 'Away'로 설정
        last_number = None

        for i in range(0, len(lineup_data), 4):
            number = lineup_data[i]
            name = lineup_data[i + 1]
            position = lineup_data[i + 2]
            bats_throws = lineup_data[i + 3]

            # 번호가 '1'로 재시작하면 Home/Away 토글
            if number == '1' and last_number is not None:
                home_away_toggle = 'Away' if home_away_toggle == 'Home' else 'Home'

            player_data = pd.DataFrame({
                'Name': [name],
                'Position': [position],
                'Bats/Throws': [bats_throws],
                'Home/Away': [home_away_toggle]
            })
            lineup_df = pd.concat([lineup_df, player_data], ignore_index=True)
            last_number = number

        for i in range(len(players_num_lineup)):
            number = players_num_lineup[i]
            lineup_df['Player Number'][i] = number
            
        # Starting lineup에서 Position 연결
        positions = ['좌익수', '중견수', '우익수', '1루수', '2루수', '유격수', '3루수','포수']
        positions_num_home = {}
        positions_num_away = {}

        df_positions = lineup_df[(lineup_df['Position'].isin(positions))]
        i = 0
        j = 0

        for home_player_position in list(df_positions[df_positions['Home/Away'] == 'Home']['Position']):
            positions_num_home[home_player_position] = list(df_positions[df_positions['Home/Away'] == 'Home']['Player Number'])[i]
            i = i+1
        for away_player_position in list(df_positions[df_positions['Home/Away'] == 'Away']['Position']):
            positions_num_away[away_player_position] = list(df_positions[df_positions['Home/Away'] == 'Away']['Player Number'])[j]
            j = j+1
        
        # Home/Away 수비 RAA 산출
        home_outfield_raa_sum, home_infield_raa_sum, home_catcher_raa_sum = calculate_raa_sums(positions_num_home, outfield_recode, infield_recode, catcher_recode)
        away_outfield_raa_sum, away_infield_raa_sum, away_catcher_raa_sum = calculate_raa_sums(positions_num_away, outfield_recode, infield_recode, catcher_recode)
        
        # Gamelog 생성
        for i in range(int(len(data)/12)):
            row = {}
            
            row['날짜'] = date
            row['경기장'] = stadium
            row['Pitcher Number'] = players_num[2*i+20]
            row['Hitter Number'] = players_num[2*i + 21]
            if data[12*i] != '':
                row['회'] = data[12*i]
                inning = data[12*i]
            else:
                row['회'] = inning
                
            # 수비 RAA 지정
            if row['회'][-1] == '초':
                row['내야 수비 RAA'] = home_infield_raa_sum
                row['외야 수비 RAA'] = home_outfield_raa_sum
                row['포수 수비 RAA'] = home_catcher_raa_sum
            else:
                row['내야 수비 RAA'] = away_infield_raa_sum
                row['외야 수비 RAA'] = away_outfield_raa_sum
                row['포수 수비 RAA'] = away_catcher_raa_sum 

            for j in range(11):
                index = i*12 + 1 + j
                row[columns[j+8]] = data[index]

            df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    
    return df

In [4]:
gamelog_2023 = gamelog_scraper(2023)

skip
skip
skip
4
5
6
7
8
9
10
11
skip
잠실 04-01


C:\Users\kimhw\AppData\Local\Temp\ipykernel_28724\281215043.py:49: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  raa_include_framing = float(players_recode[players_recode['Position'] == 'C']['Fielding Runs Above Average/144 Games, 평균 대비 수비 득점 기여 (144경기 기준)'])
C:\Users\kimhw\AppData\Local\Temp\ipykernel_28724\281215043.py:50: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  raa_framing_144 = (float(players_recode[players_recode['Position'] == 'C']['Fielding Runs (Framing), 프레이밍 관련 득점 기여'])*144) / float(players_recode[players_recode['Position'] == 'C']['Games Played, 출장'])
C:\Users\kimhw\AppData\Local\Temp\ipykernel_28724\2171858303.py:160: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or a

고척 04-01


C:\Users\kimhw\AppData\Local\Temp\ipykernel_28724\281215043.py:49: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  raa_include_framing = float(players_recode[players_recode['Position'] == 'C']['Fielding Runs Above Average/144 Games, 평균 대비 수비 득점 기여 (144경기 기준)'])
C:\Users\kimhw\AppData\Local\Temp\ipykernel_28724\281215043.py:50: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  raa_framing_144 = (float(players_recode[players_recode['Position'] == 'C']['Fielding Runs (Framing), 프레이밍 관련 득점 기여'])*144) / float(players_recode[players_recode['Position'] == 'C']['Games Played, 출장'])


문학 04-01


...

C:\Users\kimhw\AppData\Local\Temp\ipykernel_28724\281215043.py:49: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  raa_include_framing = float(players_recode[players_recode['Position'] == 'C']['Fielding Runs Above Average/144 Games, 평균 대비 수비 득점 기여 (144경기 기준)'])
C:\Users\kimhw\AppData\Local\Temp\ipykernel_28724\281215043.py:50: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  raa_framing_144 = (float(players_recode[players_recode['Position'] == 'C']['Fielding Runs (Framing), 프레이밍 관련 득점 기여'])*144) / float(players_recode[players_recode['Position'] == 'C']['Games Played, 출장'])


수원 11-11


C:\Users\kimhw\AppData\Local\Temp\ipykernel_28724\281215043.py:49: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  raa_include_framing = float(players_recode[players_recode['Position'] == 'C']['Fielding Runs Above Average/144 Games, 평균 대비 수비 득점 기여 (144경기 기준)'])
C:\Users\kimhw\AppData\Local\Temp\ipykernel_28724\281215043.py:50: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  raa_framing_144 = (float(players_recode[players_recode['Position'] == 'C']['Fielding Runs (Framing), 프레이밍 관련 득점 기여'])*144) / float(players_recode[players_recode['Position'] == 'C']['Games Played, 출장'])


잠실 11-13


C:\Users\kimhw\AppData\Local\Temp\ipykernel_28724\281215043.py:49: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  raa_include_framing = float(players_recode[players_recode['Position'] == 'C']['Fielding Runs Above Average/144 Games, 평균 대비 수비 득점 기여 (144경기 기준)'])
C:\Users\kimhw\AppData\Local\Temp\ipykernel_28724\281215043.py:50: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  raa_framing_144 = (float(players_recode[players_recode['Position'] == 'C']['Fielding Runs (Framing), 프레이밍 관련 득점 기여'])*144) / float(players_recode[players_recode['Position'] == 'C']['Games Played, 출장'])


In [5]:
gamelog_2023

,날짜,경기장,Pitcher Number,Hitter Number,회,내야 수비 RAA,외야 수비 RAA,포수 수비 RAA,투수 이름,타자 이름,투구수,결과,이전상황,이후상황,LEV,REs,REa,WPe,WPa
0,04-01,잠실,13941,14571,1회초,19.37,-0.87,2.996907,알칸타라,안권수,3 (0-2),헛스윙 삼진,무사 0:0,1사 0:0,0.87,0.523,-0.257,52.2%,-0.022
1,04-01,잠실,13941,10014,1회초,19.37,-0.87,2.996907,알칸타라,안치홍,6 (3-2),유격수 땅볼 아웃,1사 0:0,2사 0:0,0.62,0.266,-0.167,53.8%,-0.016
2,04-01,잠실,13941,15152,1회초,19.37,-0.87,2.996907,알칸타라,렉스,2 (1-0),우익수 방면 1루타,2사 0:0,2사 1루 0:0,0.40,0.099,0.116,52.6%,0.012
3,04-01,잠실,13941,13145,1회초,19.37,-0.87,2.996907,알칸타라,한동희,4 (2-1),좌익수 플라이 아웃,2사 1루 0:0,이닝종료 0:0,0.79,0.215,-0.215,54.8%,-0.022
4,04-01,잠실,14627,10195,1회말,-20.07,20.20,-1.126330,스트레일리,정수빈,4 (1-2),좌익수 방면 1루타,무사 0:0,무사 1루 0:0,0.87,0.523,0.417,58.3%,0.035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73091,11-13,잠실,15089,10387,8회말,14.46,13.22,1.826552,박영현,박동원,4 (2-1),투수 땅볼 아웃,2사 2:6,이닝종료 2:6,0.02,0.099,-0.099,98.6%,-0.001
73092,11-13,잠실,,,9회초,5.49,38.13,2.037500,,,,투수 함덕주 : 투수 고우석 (으)로 교체,,,,,,,
73093,11-13,잠실,12907,10469,9회초,5.49,38.13,2.037500,고우석,박경수,6 (3-2),3루수 파울플라이 아웃,무사 2:6,1사 2:6,0.34,0.523,-0.257,99.5%,-0.009
73094,11-13,잠실,12907,12943,9회초,5.49,38.13,2.037500,고우석,조용호,4 (1-2),헛스윙 삼진,1사 2:6,2사 2:6,0.17,0.266,-0.167,99.9%,-0.004


In [6]:
batting_bat_2023 = recode_scraper('batting','bat',2023)

In [7]:
batting_bat_2023['Hitter Number'] = batting_bat_2023['Number']
batting_bat_2023.columns

Index(['Number', '이름', 'Position', 'Plate Appearance, 타석',
       'Balls In Play, 인플레이 타구',
       'Batting Average on Balls In Play, 인플레이 타구의 안타 비율', '땅볼%', '내야 뜬볼%',
       '외야 뜬볼%', '뜬볼%', '라인드라이브%', '땅볼 / 뜬볼', '홈런 / 뜬볼%', '내야안타%', '실책 출루',
       '포스 아웃 출루', '번트 안타', '번트 아웃', '번트 타율', '번트 시도', '번트 안타 성공율, 번트안타/번트시도',
       'Hitter Number'],
      dtype='object')

In [8]:
batting_bat_2023 = batting_bat_2023[['Hitter Number','Batting Average on Balls In Play, 인플레이 타구의 안타 비율','땅볼%','내야 뜬볼%',
                                     '외야 뜬볼%','뜬볼%','라인드라이브%','홈런 / 뜬볼%','내야안타%']]
batting_bat_2023

,Hitter Number,"Batting Average on Balls In Play, 인플레이 타구의 안타 비율",땅볼%,내야 뜬볼%,외야 뜬볼%,뜬볼%,라인드라이브%,홈런 / 뜬볼%,내야안타%
0,13161,0.191,72.7,0.0,22.7,22.7,4.5,20.0,0.0
1,13056,0.312,69.3,0.7,27.7,28.5,2.2,0.0,25.6
2,13006,0.400,68.8,2.2,26.9,29.0,2.2,0.0,19.4
3,10119,0.226,67.7,0.0,29.0,29.0,3.2,0.0,14.3
4,13195,0.286,70.0,10.0,20.0,30.0,0.0,0.0,25.0
...,...,...,...,...,...,...,...,...,...
206,10106,0.308,29.9,7.2,59.8,67.0,3.1,11.1,3.6
207,15539,0.260,29.6,9.5,57.4,66.9,3.6,8.4,5.9
208,10108,0.290,29.8,5.8,62.5,68.3,1.9,2.8,3.2
209,11213,0.331,29.1,9.0,59.2,68.2,2.7,7.7,4.8


In [9]:
batting_direction_2023 = recode_scraper('batting','direction',2023)

In [10]:
batting_direction_2023['Hitter Number'] = batting_direction_2023['Number']
batting_direction_2023.columns

Index(['Number', '이름', 'Position', 'Plate Appearance, 타석', '전체 좌측 타구 비율',
       '전체 좌중앙 타구 비율', '전체 중앙 타구 비율', '전체 우중앙 타구 비율', '전체 우측 타구 비율',
       '전체 당겨친 타구 비율', '전체 밀어친 타구 비율', '전체 좌측 타구 개수', '전체 좌중앙 타구 개수',
       '전체 중앙 타구 개수', '전체 우중앙 타구 개수', '전체 우측 타구 개수', '전체 당겨친 타구 개수',
       '전체 밀어친 타구 개수', '전체 좌측 안타 개수', '전체 좌중앙 안타 개수', '전체 중앙 안타 개수',
       '전체 우중앙 안타 개수', '전체 우측 안타 개수', '전체 당겨친 안타 개수', '전체 밀어친 안타 개수',
       '좌측 타구 타율', '좌중앙 타구 타율', '중앙 타구 타율', '우중앙 타구 타율', '우측 타구 타율',
       '당겨친 타구 타율', '밀어친 타구 타율',
       'Batting Average on Balls In Play, 인플레이 타구의 안타 비율', 'Hitter Number'],
      dtype='object')

In [11]:
batting_direction_2023 = batting_direction_2023[['Hitter Number','전체 좌측 타구 비율','전체 좌중앙 타구 비율','전체 중앙 타구 비율',
                                               '전체 우중앙 타구 비율','전체 우측 타구 비율']]
batting_direction_2023

,Hitter Number,전체 좌측 타구 비율,전체 좌중앙 타구 비율,전체 중앙 타구 비율,전체 우중앙 타구 비율,전체 우측 타구 비율
0,15132,12.8,23.1,23.1,20.5,20.5
1,10810,22.2,16.7,22.2,16.7,22.2
2,12606,14.7,23.5,20.6,14.7,26.5
3,10804,14.9,16.7,18.6,23.3,26.5
4,14591,12.7,15.8,11.5,27.3,32.7
...,...,...,...,...,...,...
206,12549,29.2,37.5,25.0,8.3,0.0
207,10684,23.5,21.6,23.5,23.5,7.8
208,10913,16.0,16.0,16.0,20.0,32.0
209,10082,15.8,18.4,7.9,31.6,26.3


In [12]:
batting_pitch_2023 = recode_scraper('batting','pitch',2023)

In [13]:
batting_pitch_2023['Hitter Number'] = batting_pitch_2023['Number']
batting_pitch_2023.columns

Index(['Number', '이름', 'Position', 'Plate Appearance, 타석',
       'Strike%, 전체 투구 대비 스트라이크', 'Called Strike%, 전체 투구 대비 루킹 스트라이크%',
       'Whiff%, 전체 투구 대비 헛스윙 스트라이크%', 'CSW%, 전체 투구 대비 루킹+헛스윙 스트라이크%',
       'Swing%, 스윙 비율', '스윙 대비 콘택트 비율', '스윙 대비 헛스윙 비율', '초구 스트라이크 비율',
       '초구 스윙 비율', '투스트라이크 카운트 투구 대비 삼진 결정 비율', 'Strike Zone%, 존 안에 들어온 투구 비율',
       'Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율',
       'Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율',
       'Out Zone%, 존 밖에 들어온 투구 비율', 'Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율',
       'Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율',
       'Meatball Zone%, 존 한가운데 들어온 투구 비율',
       'Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율',
       'Shadow Zone%, 쉐도우존에 들어온 투구 비율', 'Looking Strike Out, 루킹 삼진',
       'Swing Strike Out, 헛스윙 삼진', 'Looking Strike Out%, 루킹 삼진 비율',
       'Hitter Number'],
      dtype='object')

In [14]:
batting_pitch_2023 = batting_pitch_2023[['Hitter Number','Strike%, 전체 투구 대비 스트라이크', 'Called Strike%, 전체 투구 대비 루킹 스트라이크%',
       'Whiff%, 전체 투구 대비 헛스윙 스트라이크%', 'CSW%, 전체 투구 대비 루킹+헛스윙 스트라이크%',
       'Swing%, 스윙 비율', '스윙 대비 콘택트 비율', '스윙 대비 헛스윙 비율', '초구 스트라이크 비율',
       '초구 스윙 비율', '투스트라이크 카운트 투구 대비 삼진 결정 비율', 'Strike Zone%, 존 안에 들어온 투구 비율',
       'Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율',
       'Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율',
       'Out Zone%, 존 밖에 들어온 투구 비율', 'Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율',
       'Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율',
       'Meatball Zone%, 존 한가운데 들어온 투구 비율',
       'Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율',
       'Shadow Zone%, 쉐도우존에 들어온 투구 비율', 'Looking Strike Out%, 루킹 삼진 비율']]
batting_pitch_2023

,Hitter Number,"Strike%, 전체 투구 대비 스트라이크","Called Strike%, 전체 투구 대비 루킹 스트라이크%","Whiff%, 전체 투구 대비 헛스윙 스트라이크%","CSW%, 전체 투구 대비 루킹+헛스윙 스트라이크%","Swing%, 스윙 비율",스윙 대비 콘택트 비율,스윙 대비 헛스윙 비율,초구 스트라이크 비율,초구 스윙 비율,...,"Strike Zone%, 존 안에 들어온 투구 비율","Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율","Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율","Out Zone%, 존 밖에 들어온 투구 비율","Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율","Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율","Meatball Zone%, 존 한가운데 들어온 투구 비율","Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율","Shadow Zone%, 쉐도우존에 들어온 투구 비율","Looking Strike Out%, 루킹 삼진 비율"
0,10913,75.6,10.1,14.3,24.4,68.1,77.8,22.2,71.1,10.5,...,52.9,84.1,68.3,47.1,44.6,32.1,3.4,75.0,20.2,10.0
1,15443,71.2,23.7,7.9,31.6,48.6,83.7,16.3,70.2,4.3,...,42.4,50.7,46.7,57.6,44.1,33.3,4.5,37.5,19.8,37.5
2,10119,71.1,25.7,10.5,36.2,49.3,77.3,22.7,64.3,2.4,...,46.1,52.9,50.0,53.9,39.0,22.0,4.6,85.7,15.8,11.1
3,14716,71.0,18.5,9.5,27.9,56.4,83.2,16.8,63.8,4.8,...,48.5,68.3,64.6,51.5,37.8,22.9,4.6,83.3,17.9,26.1
4,12606,71.0,8.1,18.8,26.9,68.3,70.9,29.1,68.6,15.7,...,43.5,82.7,63.0,56.5,47.6,29.5,5.9,90.9,20.4,7.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,10643,58.7,14.6,12.1,26.7,48.4,73.6,26.4,58.4,10.4,...,41.7,68.6,55.4,58.3,26.6,15.3,5.5,78.5,19.5,23.0
207,12546,58.2,18.2,6.3,24.5,44.8,85.3,14.7,54.2,3.9,...,44.7,65.5,58.6,55.3,19.4,13.7,6.1,78.4,19.4,32.5
208,14888,58.2,15.1,9.4,24.6,47.2,79.1,20.9,52.7,7.1,...,41.2,68.8,59.7,58.8,25.0,15.4,5.5,74.3,19.6,21.5
209,10840,57.8,17.1,8.1,25.2,44.4,79.7,20.3,50.7,8.0,...,39.1,63.5,54.8,60.9,26.0,18.4,5.0,75.0,20.8,37.5


In [52]:
batting_balltype_2023 = recode_scraper('batting','ballType',2023)

In [53]:
batting_balltype_2023

,Number,이름,Position,"Plate Appearance, 타석",구종가치 (투심),구종가치 (포심),구종가치 (커터),구종가치 (커브),구종가치 (슬라이더),구종가치 (체인지업),...,구종별 피장타율 (투심),구종별 피장타율 (포심),구종별 피장타율 (커터),구종별 피장타율 (커브),구종별 피장타율 (슬라이더),구종별 피장타율 (체인지업),구종별 피장타율 (싱커),구종별 피장타율 (포크볼),구종별 피장타율 (너클볼),구종별 피장타율 (기타)
0,11162,나성범,RF,253,0.7,16.9,0.5,3.3,-3.6,2.6,...,0.444,0.831,1.000,0.800,0.365,0.700,1.500,0.579,,
1,10892,구자욱,RF,515,-2.3,15.3,0.3,2.5,2.2,-0.7,...,0.353,0.571,0.545,0.442,0.450,0.377,0.000,0.548,,
2,10189,박건우,RF,533,2.1,14.5,-2.3,2.5,-2.9,1.2,...,0.579,0.585,0.100,0.514,0.354,0.414,0.875,0.320,,
3,10261,손아섭,DH,609,-0.9,13.0,0.8,-1.8,0.1,1.0,...,0.407,0.519,0.563,0.306,0.410,0.410,0.750,0.333,,
4,10106,최정,3B,552,0.1,12.6,-1.1,0.0,1.8,7.3,...,0.375,0.601,0.000,0.415,0.519,0.629,1.000,0.318,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,14220,서호철,3B,435,4.1,-14.7,0.7,-1.7,2.5,0.9,...,0.727,0.294,0.600,0.305,0.500,0.375,0.625,0.391,,0.000
207,10180,이원석,1B,345,-0.4,-15.2,-0.6,-1.0,-2.4,1.9,...,0.368,0.221,0.000,0.241,0.352,0.500,0.286,0.286,,
208,13137,정은원,2B,459,-0.4,-15.6,-1.9,-2.6,1.3,-6.5,...,0.375,0.305,0.000,0.138,0.378,0.077,0.250,0.148,,0.000
209,10320,김상수,SS,512,-2.0,-16.5,-1.4,2.9,1.4,2.1,...,0.297,0.306,0.250,0.368,0.447,0.382,0.286,0.063,,


In [54]:
batting_balltype_2023['Hitter Number'] = batting_balltype_2023['Number']
batting_balltype_2023.columns

Index(['Number', '이름', 'Position', 'Plate Appearance, 타석', '구종가치 (투심)',
       '구종가치 (포심)', '구종가치 (커터)', '구종가치 (커브)', '구종가치 (슬라이더)', '구종가치 (체인지업)',
       '구종가치 (싱커)', '구종가치 (포크볼)', '구종가치 (너클볼)', '구종가치 (기타)', '구종가치/100 (투심)',
       '구종가치/100 (포심)', '구종가치/100 (커터)', '구종가치/100 (커브)', '구종가치/100 (슬라이더)',
       '구종가치/100 (체인지업)', '구종가치/100 (싱커)', '구종가치/100 (포크볼)', '구종가치/100 (너클볼)',
       '구종가치/100 (기타)', '평균구속 (투심)', '평균구속 (포심)', '평균구속 (커터)', '평균구속 (커브)',
       '평균구속 (슬라이더)', '평균구속 (체인지업)', '평균구속 (싱커)', '평균구속 (포크볼)', '평균구속 (너클볼)',
       '평균구속 (기타)', '구종별 구사율 (투심)', '구종별 구사율 (포심)', '구종별 구사율 (커터)',
       '구종별 구사율 (커브)', '구종별 구사율 (슬라이더)', '구종별 구사율 (체인지업)', '구종별 구사율 (싱커)',
       '구종별 구사율 (포크볼)', '구종별 구사율 (너클볼)', '구종별 구사율 (기타)', '구종별 투구 수 (투심)',
       '구종별 투구 수 (포심)', '구종별 투구 수 (커터)', '구종별 투구 수 (커브)', '구종별 투구 수 (슬라이더)',
       '구종별 투구 수 (체인지업)', '구종별 투구 수 (싱커)', '구종별 투구 수 (포크볼)', '구종별 투구 수 (너클볼)',
       '구종별 투구 수 (기타)', '구종별 피안타율 (투심)', '구종별 피안타율 (포심)', '구종별 피안타율 (커터)',
       '구종별 피안타율

In [55]:
batting_balltype_two_seam = list(batting_balltype_2023['구종가치 (투심)'])
two_seam_list = pd.to_numeric(batting_balltype_two_seam, errors='coerce')
two_seam_list

array([ 0.7, -2.3,  2.1, -0.9,  0.1, -0.6,  2.2,  0.4,  0. , -1. ,  2.6,
       -3.6,  2.4, -1.9, -0.2,  3.3, -1.4,  1.6, -3. ,  0.3,  1.2,  0.1,
       -2.4,  1.6,  4.2,  6.1, -0.2,  1.7,  1.3,  0.7, -0.9,  2.8,  nan,
        0.1,  4.4,  1.2, -2. , -0.5, -1.5, -0.8, -0.3, -0.7, -0.8, -0.5,
       -0.9,  0.1, -0.6, -2.2, -2.5,  0.1, -0.4,  0.6, -3.4, -1.7, -0.5,
       -0.8, -1.1, -0.7,  0.8,  0. , -0.1, -0.1,  0.6, -1.9,  0.3, -0.6,
       -5.8, -0.5,  1.6, -0.5,  0.2, -0.9, -0.6,  1.3,  1.1, -1.2, -1.8,
       -1.1, -0.1,  0.6, -0.1, -3.7,  1.2,  0.7, -0.6, -0.2,  1.2, -0.4,
       -1.1,  1.2, -1.6, -0.1, -1.1, -1.3,  1.1,  3.5, -0.3, -0.5,  0.5,
       -0.3, -1.4,  0. , -1.1,  0.6, -0.3, -0.3,  2.6, -0.8, -0.2, -0.9,
        1.5, -1.2, -1.1, -1.2, -0.8, -0.8, -0.3, -0.2, -0.1, -0.5,  0.9,
       -0.3, -1.8, -0.9,  0.4,  0.7,  0.3, -1.1,  0.3, -0.7, -0.7, -0.1,
       -0.5, -0.3,  1. , -0.4,  0.2,  0.2, -2.4,  0.7, -1.3,  0.3,  2.6,
       -0.7, -0.2, -0.2, -0.7,  1.2,  7.2,  1.2,  0

In [56]:
batting_balltype_two_seam_throw = list(batting_balltype_2023['구종별 투구 수 (투심)'])
two_seam_throw_list = pd.to_numeric(batting_balltype_two_seam_throw, errors='coerce')
type(two_seam_throw_list)

numpy.ndarray

In [57]:
# 10개 이하로 투구는 Nan 처리

import numpy as np

two_seam_throw_list[two_seam_throw_list < 10] = np.nan

In [58]:
two_seam_throw_list

array([ 45.,  83., 113., 122., 115., 107.,  85.,  94., 110.,  78., 100.,
        95.,  51.,  94., 116., 100.,  23.,  82., 110.,  23.,  22.,  79.,
       104.,  85., 176., 106., 100.,  40.,  18.,  28.,  19., 103.,  nan,
        nan, 110.,  19.,  52.,  nan,  58.,  36.,  69.,  40.,  72.,  nan,
       100.,  48.,  74.,  16.,  94.,  nan,  nan,  99.,  80.,  41.,  35.,
       123.,  63.,  20., 137.,  nan,  nan,  nan,  nan,  15., 142.,  nan,
        79.,  18.,  65.,  11., 100.,  15.,  16.,  73.,  64.,  11.,  40.,
        97.,  37.,  nan,  nan,  72.,  37.,  73.,  20., 105.,  16.,  13.,
        14.,  80.,  41.,  nan,  20.,  54.,  18., 112.,  nan,  nan,  nan,
        19.,  15.,  28.,  26.,  33.,  nan,  nan,  30.,  17.,  36.,  22.,
        nan,  16.,  21.,  20.,  nan,  11.,  nan,  nan,  nan,  18.,  nan,
        nan,  50.,  14.,  22.,  16., 106.,  59.,  33.,  18.,  nan,  nan,
        12.,  nan,  38.,  28.,  23.,  17.,  82.,  74.,  16., 128.,  50.,
        14.,  nan,  19.,  nan,  18., 108.,  23.,  9

In [59]:
batting_balltype_2023['구종가치/100 (투심)'] = two_seam_list / two_seam_throw_list *100

In [60]:
batting_balltype_2023['구종가치/100 (투심)']

0      1.555556
1     -2.771084
2      1.858407
3     -0.737705
4      0.086957
         ...   
206    5.061728
207   -0.444444
208   -0.416667
209   -1.408451
210   -0.909091
Name: 구종가치/100 (투심), Length: 211, dtype: float64

In [61]:
def balltype_value_divider(data, type):
    
    if type == 'batting':
        data['Hitter Number'] = data['Number']

        balltype_list = ['구종가치 (투심)',
               '구종가치 (포심)', '구종가치 (커터)', '구종가치 (커브)', '구종가치 (슬라이더)', '구종가치 (체인지업)',
               '구종가치 (싱커)', '구종가치 (포크볼)']
        balltype_throw_list = ['구종별 투구 수 (투심)',
           '구종별 투구 수 (포심)', '구종별 투구 수 (커터)', '구종별 투구 수 (커브)', '구종별 투구 수 (슬라이더)',
           '구종별 투구 수 (체인지업)', '구종별 투구 수 (싱커)', '구종별 투구 수 (포크볼)']
        balltype_100_list = ['구종가치/100 (투심)',
           '구종가치/100 (포심)', '구종가치/100 (커터)', '구종가치/100 (커브)', '구종가치/100 (슬라이더)',
           '구종가치/100 (체인지업)', '구종가치/100 (싱커)', '구종가치/100 (포크볼)']
    elif type == 'pitching':
        data['Pitcher Number'] = data['Number']
        
        balltype_list = ['2-Seamer Fastball Pitch Value, 구종가치 (투심)',
       '4-Seamer Fastball Pitch Value, 구종가치 (포심)',
       'Cutter Pitch Value, 구종가치 (커터)', 'Curve Pitch Value, 구종가치 (커브)',
       'Slider Pitch Value, 구종가치 (슬라이더)', 'Changeup Pitch Value, 구종가치 (체인지업)',
       'Sinker Pitch Value, 구종가치 (싱커)', 'Forkball Pitch Value, 구종가치 (포크볼)']
        balltype_throw_list = ['2-Seamer Fastball Pitch Count, 구종별 투구 수 (투심)',
       '4-Seamer Fastball Pitch Count, 구종별 투구 수 (포심)',
       'Cutter Pitch Count, 구종별 투구 수 (커터)', 'Curve Pitch Count, 구종별 투구 수 (커브)',
       'Slider Pitch Count, 구종별 투구 수 (슬라이더)',
       'Changeup Pitch Count, 구종별 투구 수 (체인지업)',
       'Sinker Pitch Count, 구종별 투구 수 (싱커)',
       'Forkball Pitch Count, 구종별 투구 수 (포크볼)']
        balltype_speed_list = ['2-Seamer Fastball Velocity, 평균구속 (투심)','4-Seamer Fastball Velocity, 평균구속 (포심)',
                              'Cutter Velocity, 평균구속 (커터)', 'Curve Velocity, 평균구속 (커브)',
                               'Slider Velocity, 평균구속 (슬라이더)','Changeup Velocity, 평균구속 (체인지업)',
                              'Sinker Velocity, 평균구속 (싱커)','Forkball Velocity, 평균구속 (포크볼)']
        balltype_100_list = ['2-Seamer Fastball Pitch Value per 100, 구종가치/100 (투심)',
       '4-Seamer Fastball Pitch Value per 100, 구종가치/100 (포심)',
       'Cutter Pitch Value per 100, 구종가치/100 (커터)',
       'Curve Pitch Value per 100, 구종가치/100 (커브)',
       'Slider Pitch Value per 100, 구종가치/100 (슬라이더)',
       'Changeup Pitch Value per 100, 구종가치/100 (체인지업)',
       'Sinker Pitch Value per 100, 구종가치/100 (싱커)',
       'Forkball Pitch Value per 100, 구종가치/100 (포크볼)']
        

    for balltype, balltype_throw, balltype_100 in zip(balltype_list, balltype_throw_list, balltype_100_list):
        balltype_value_list = [float(value) if value != '' else np.nan for value in data[balltype]]
        balltype_throw_num_list = [float(value) if value != '' else np.nan for value in data[balltype_throw]]
        balltype_throw_num_list = np.array(balltype_throw_num_list)
        balltype_throw_num_list[balltype_throw_num_list < 10] = np.nan
        
        data[balltype_100] = balltype_value_list / balltype_throw_num_list * 100
    
    if type == 'batting':
        data = data[['Hitter Number'] + balltype_100_list]
    elif type == 'pitching':
        for balltype_speed in balltype_speed_list:
            data[balltype_speed] = [float(value) if value != '' else np.nan for value in data[balltype_speed]]
        
        data = data[['Pitcher Number'] + balltype_speed_list + balltype_100_list]
    
    return data
    
    
batting_balltype_2023 = balltype_value_divider(batting_balltype_2023, 'batting')
batting_balltype_2023

,Hitter Number,구종가치/100 (투심),구종가치/100 (포심),구종가치/100 (커터),구종가치/100 (커브),구종가치/100 (슬라이더),구종가치/100 (체인지업),구종가치/100 (싱커),구종가치/100 (포크볼)
0,11162,1.555556,3.894009,1.562500,3.666667,-1.894737,2.765957,NaN,1.358025
1,10892,-2.771084,1.879607,0.967742,1.243781,0.435644,-0.292887,-1.794872,1.933702
2,10189,1.858407,1.711924,-5.750000,1.373626,-0.590631,0.512821,7.727273,-0.800000
3,10261,-0.737705,1.326531,1.481481,-0.837209,0.020367,0.492611,2.307692,-0.239521
4,10106,0.086957,1.580928,-3.437500,0.000000,0.339623,2.401316,7.096774,-0.375940
...,...,...,...,...,...,...,...,...,...
206,14220,5.061728,-2.184250,3.888889,-0.890052,0.786164,0.656934,4.347826,0.106383
207,10180,-0.444444,-2.962963,-6.000000,-0.675676,-0.754717,1.328671,-2.000000,-1.216216
208,13137,-0.416667,-1.822430,-4.871795,-1.793103,0.310263,-3.299492,-2.000000,-1.744966
209,10320,-1.408451,-1.552211,-3.589744,1.812500,0.299786,1.390728,-0.789474,-2.298851


In [62]:
batting_player_data = pd.merge(batting_bat_2023,batting_direction_2023)
batting_player_data = pd.merge(batting_pitch_2023,batting_player_data)
batting_player_data = pd.merge(batting_balltype_2023,batting_player_data)
batting_player_data

,Hitter Number,구종가치/100 (투심),구종가치/100 (포심),구종가치/100 (커터),구종가치/100 (커브),구종가치/100 (슬라이더),구종가치/100 (체인지업),구종가치/100 (싱커),구종가치/100 (포크볼),"Strike%, 전체 투구 대비 스트라이크",...,외야 뜬볼%,뜬볼%,라인드라이브%,홈런 / 뜬볼%,내야안타%,전체 좌측 타구 비율,전체 좌중앙 타구 비율,전체 중앙 타구 비율,전체 우중앙 타구 비율,전체 우측 타구 비율
0,11162,1.555556,3.894009,1.562500,3.666667,-1.894737,2.765957,NaN,1.358025,62.3,...,49.5,56.8,2.6,16.7,3.7,18.8,15.1,15.1,22.0,29.0
1,10892,-2.771084,1.879607,0.967742,1.243781,0.435644,-0.292887,-1.794872,1.933702,62.8,...,50.7,56.2,4.2,5.2,7.9,12.5,16.8,17.6,25.3,27.9
2,10189,1.858407,1.711924,-5.750000,1.373626,-0.590631,0.512821,7.727273,-0.800000,60.7,...,54.4,57.2,2.8,5.3,6.8,26.0,23.7,18.7,16.4,15.2
3,10261,-0.737705,1.326531,1.481481,-0.837209,0.020367,0.492611,2.307692,-0.239521,63.0,...,32.2,36.1,3.7,2.8,14.4,15.6,22.5,17.6,23.8,20.5
4,10106,0.086957,1.580928,-3.437500,0.000000,0.339623,2.401316,7.096774,-0.375940,60.8,...,59.8,67.0,3.1,11.1,3.6,39.1,25.9,11.9,10.6,12.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,14220,5.061728,-2.184250,3.888889,-0.890052,0.786164,0.656934,4.347826,0.106383,67.8,...,42.1,48.9,2.5,2.9,12.3,26.0,24.0,18.6,14.6,16.9
207,10180,-0.444444,-2.962963,-6.000000,-0.675676,-0.754717,1.328671,-2.000000,-1.216216,60.6,...,50.2,61.0,2.5,1.4,4.0,27.5,21.2,15.7,20.8,14.8
208,13137,-0.416667,-1.822430,-4.871795,-1.793103,0.310263,-3.299492,-2.000000,-1.744966,58.8,...,44.2,50.5,2.5,1.2,7.0,15.3,16.2,15.0,26.8,26.8
209,10320,-1.408451,-1.552211,-3.589744,1.812500,0.299786,1.390728,-0.789474,-2.298851,61.9,...,44.0,49.2,3.4,1.6,8.3,18.8,24.2,20.6,21.9,14.6


In [63]:
def split_finder(player_num,type):
    url = 'http://statiz.sporki.com/player'

    params = {
        'm':'playerinfo',
        'p_no':player_num
    }

    resp = request('GET',url=url, params=params)
    dom = BeautifulSoup(resp.text,'html.parser')
    
    print(player_num)
    
    text = dom.find_all(attrs={'class':'con'})[0].text[-5:-1]
    
    if type == 'pitching':
        pitching_type = text[0:2]
        
        if pitching_type == '우투':
            return 0.0
        
        elif pitching_type == '좌투':
            return 1.0
        
        elif pitching_type == '우언':
            return 0.25
        
        elif pitching_type == '좌언':
            return 0.75
        
    elif type == 'batting':
        batting_type = text[2:4]
        
        if batting_type == '우타':
            return 0.0
        
        elif batting_type == '좌타':
            return 1.0
        
        elif batting_type == '양타':
            return 0.5

In [64]:
batting_split_data = []

for id in list(batting_player_data['Hitter Number']):
    batting_split_num = split_finder(id,'batting')
    batting_split_data.append(batting_split_num)

batting_player_data['투타'] = batting_split_data

11162
10892
10189
10261
10106
10266
14125
14137
14996
15056
15530
14888
10232
11339
10344
10470
11148
12905
12922
14591
13154
10407
14757
12906
12546
15144
11137
14147
15652
15651
15647
10182
15132
15519
10165
12936
11212
14221
11306
12560
10174
10659
11172
13164
10475
10253
11165
13104
10753
13161
10810
11215
10387
14124
11397
13112
11135
11098
10008
12606
13093
11246
14863
13676
10195
14590
10815
10840
11233
12613
10187
10684
13081
14785
14160
13195
12943
14618
14603
13194
14181
10891
10108
10804
12503
11087
12548
13128
11344
10238
11198
14796
15035
10400
14699
11099
12970
12622
12931
11147
13113
13096
13261
12916
12549
10119
14707
11407
13151
12562
15503
10480
13250
14214
12988
13006
14172
14126
14117
14704
14133
12526
11333
12567
12941
11298
10014
10249
11329
14170
14771
10913
11268
14851
12585
10835
14738
11422
10894
14571
11214
11190
14742
10082
14175
12540
15443
14580
15532
10322
15539
11255
15152
15541
14114
12534
14642
12587
13138
13240
13168
15473
10469
11261
12522
11225
1480

In [65]:
batting_player_data

,Hitter Number,구종가치/100 (투심),구종가치/100 (포심),구종가치/100 (커터),구종가치/100 (커브),구종가치/100 (슬라이더),구종가치/100 (체인지업),구종가치/100 (싱커),구종가치/100 (포크볼),"Strike%, 전체 투구 대비 스트라이크",...,뜬볼%,라인드라이브%,홈런 / 뜬볼%,내야안타%,전체 좌측 타구 비율,전체 좌중앙 타구 비율,전체 중앙 타구 비율,전체 우중앙 타구 비율,전체 우측 타구 비율,투타
0,11162,1.555556,3.894009,1.562500,3.666667,-1.894737,2.765957,NaN,1.358025,62.3,...,56.8,2.6,16.7,3.7,18.8,15.1,15.1,22.0,29.0,1.0
1,10892,-2.771084,1.879607,0.967742,1.243781,0.435644,-0.292887,-1.794872,1.933702,62.8,...,56.2,4.2,5.2,7.9,12.5,16.8,17.6,25.3,27.9,1.0
2,10189,1.858407,1.711924,-5.750000,1.373626,-0.590631,0.512821,7.727273,-0.800000,60.7,...,57.2,2.8,5.3,6.8,26.0,23.7,18.7,16.4,15.2,0.0
3,10261,-0.737705,1.326531,1.481481,-0.837209,0.020367,0.492611,2.307692,-0.239521,63.0,...,36.1,3.7,2.8,14.4,15.6,22.5,17.6,23.8,20.5,1.0
4,10106,0.086957,1.580928,-3.437500,0.000000,0.339623,2.401316,7.096774,-0.375940,60.8,...,67.0,3.1,11.1,3.6,39.1,25.9,11.9,10.6,12.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,14220,5.061728,-2.184250,3.888889,-0.890052,0.786164,0.656934,4.347826,0.106383,67.8,...,48.9,2.5,2.9,12.3,26.0,24.0,18.6,14.6,16.9,0.0
207,10180,-0.444444,-2.962963,-6.000000,-0.675676,-0.754717,1.328671,-2.000000,-1.216216,60.6,...,61.0,2.5,1.4,4.0,27.5,21.2,15.7,20.8,14.8,0.0
208,13137,-0.416667,-1.822430,-4.871795,-1.793103,0.310263,-3.299492,-2.000000,-1.744966,58.8,...,50.5,2.5,1.2,7.0,15.3,16.2,15.0,26.8,26.8,1.0
209,10320,-1.408451,-1.552211,-3.589744,1.812500,0.299786,1.390728,-0.789474,-2.298851,61.9,...,49.2,3.4,1.6,8.3,18.8,24.2,20.6,21.9,14.6,0.0


In [66]:
batting_player_data.to_excel('batting_player_data.xlsx')

In [67]:
pitching_balltype_2023 = recode_scraper('pitching','ballType',2023)
pitching_balltype_2023 = balltype_value_divider(pitching_balltype_2023, 'pitching')
pitching_balltype_2023

,Pitcher Number,"2-Seamer Fastball Velocity, 평균구속 (투심)","4-Seamer Fastball Velocity, 평균구속 (포심)","Cutter Velocity, 평균구속 (커터)","Curve Velocity, 평균구속 (커브)","Slider Velocity, 평균구속 (슬라이더)","Changeup Velocity, 평균구속 (체인지업)","Sinker Velocity, 평균구속 (싱커)","Forkball Velocity, 평균구속 (포크볼)","2-Seamer Fastball Pitch Value per 100, 구종가치/100 (투심)","4-Seamer Fastball Pitch Value per 100, 구종가치/100 (포심)","Cutter Pitch Value per 100, 구종가치/100 (커터)","Curve Pitch Value per 100, 구종가치/100 (커브)","Slider Pitch Value per 100, 구종가치/100 (슬라이더)","Changeup Pitch Value per 100, 구종가치/100 (체인지업)","Sinker Pitch Value per 100, 구종가치/100 (싱커)","Forkball Pitch Value per 100, 구종가치/100 (포크볼)"
0,13091,NaN,153.0,NaN,129.2,142.4,136.3,NaN,NaN,NaN,2.765055,NaN,0.500000,2.142857,0.541872,NaN,NaN
1,13941,NaN,150.7,NaN,123.9,135.9,136.3,NaN,135.2,NaN,1.612686,NaN,2.638889,0.214844,NaN,NaN,1.890110
2,14788,NaN,146.0,NaN,123.8,132.3,132.9,NaN,NaN,NaN,1.425729,NaN,-0.502513,2.611111,0.612245,NaN,NaN
3,15089,NaN,145.6,NaN,NaN,127.5,126.4,NaN,NaN,NaN,2.510232,NaN,NaN,-1.081081,0.771513,NaN,NaN
4,11489,NaN,143.7,NaN,119.0,125.0,NaN,NaN,130.4,NaN,2.569832,NaN,NaN,0.851064,NaN,NaN,1.458333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,10730,NaN,NaN,NaN,118.7,NaN,121.0,129.2,NaN,NaN,NaN,NaN,-0.728643,NaN,NaN,-1.072089,NaN
198,10533,134.5,NaN,NaN,113.7,128.3,NaN,NaN,126.2,2.594595,NaN,NaN,-0.042918,0.272374,NaN,NaN,0.646766
199,13940,142.0,NaN,NaN,124.4,135.6,132.6,NaN,NaN,0.960452,NaN,NaN,-0.140351,-3.797468,-0.344828,NaN,NaN
200,14136,NaN,NaN,144.3,130.8,134.1,140.0,148.9,138.5,NaN,NaN,-3.928571,-2.500000,-0.584416,NaN,0.303030,-6.153846


In [68]:
pitching_pitch_2023 = recode_scraper('pitching','pitch',2023)

In [69]:
pitching_pitch_2023.columns = ['(P) ' + column for column in pitching_pitch_2023.columns]
pitching_pitch_2023['Pitcher Number'] = pitching_pitch_2023['(P) Number']
pitching_pitch_2023.columns

Index(['(P) Number', '(P) 이름', '(P) Position', '(P) Games, 출장',
       '(P) Strike%, 전체 투구 대비 스트라이크', '(P) Called Strike%, 전체 투구 대비 루킹 스트라이크%',
       '(P) Whiff%, 전체 투구 대비 헛스윙 스트라이크%', '(P) CSW%, 전체 투구 대비 루킹+헛스윙 스트라이크%',
       '(P) Swing%, 스윙 비율', '(P) 스윙 대비 콘택트 비율', '(P) 스윙 대비 헛스윙 비율',
       '(P) 초구 스트라이크 비율', '(P) 초구 스윙 비율', '(P) 투스트라이크 카운트 투구 대비 삼진 결정 비율',
       '(P) Strike Zone%, 존 안에 들어온 투구 비율',
       '(P) Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율',
       '(P) Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율',
       '(P) Out Zone%, 존 밖에 들어온 투구 비율',
       '(P) Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율',
       '(P) Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율',
       '(P) Meatball Zone%, 존 한가운데 들어온 투구 비율',
       '(P) Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율',
       '(P) Looking Strike Out, 루킹 삼진', '(P) Swing Strike Out, 헛스윙 삼진',
       '(P) Looking Strike Out%, 루킹 삼진 비율', 'Pitcher Number'],
      dtype='object')

In [70]:
pitching_pitch_2023 = pitching_pitch_2023[['Pitcher Number', '(P) Strike%, 전체 투구 대비 스트라이크', '(P) Called Strike%, 전체 투구 대비 루킹 스트라이크%',
       '(P) Whiff%, 전체 투구 대비 헛스윙 스트라이크%', '(P) CSW%, 전체 투구 대비 루킹+헛스윙 스트라이크%',
       '(P) Swing%, 스윙 비율', '(P) 스윙 대비 콘택트 비율', '(P) 스윙 대비 헛스윙 비율',
       '(P) 초구 스트라이크 비율', '(P) 초구 스윙 비율', '(P) 투스트라이크 카운트 투구 대비 삼진 결정 비율',
       '(P) Strike Zone%, 존 안에 들어온 투구 비율',
       '(P) Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율',
       '(P) Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율',
       '(P) Out Zone%, 존 밖에 들어온 투구 비율',
       '(P) Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율',
       '(P) Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율',
       '(P) Meatball Zone%, 존 한가운데 들어온 투구 비율',
       '(P) Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율','(P) Looking Strike Out%, 루킹 삼진 비율']]
pitching_pitch_2023

,Pitcher Number,"(P) Strike%, 전체 투구 대비 스트라이크","(P) Called Strike%, 전체 투구 대비 루킹 스트라이크%","(P) Whiff%, 전체 투구 대비 헛스윙 스트라이크%","(P) CSW%, 전체 투구 대비 루킹+헛스윙 스트라이크%","(P) Swing%, 스윙 비율",(P) 스윙 대비 콘택트 비율,(P) 스윙 대비 헛스윙 비율,(P) 초구 스트라이크 비율,(P) 초구 스윙 비율,(P) 투스트라이크 카운트 투구 대비 삼진 결정 비율,"(P) Strike Zone%, 존 안에 들어온 투구 비율","(P) Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율","(P) Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율","(P) Out Zone%, 존 밖에 들어온 투구 비율","(P) Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율","(P) Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율","(P) Meatball Zone%, 존 한가운데 들어온 투구 비율","(P) Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율","(P) Looking Strike Out%, 루킹 삼진 비율"
0,11308,72.4,19.7,10.9,30.6,57.5,80.6,19.4,72.2,7.8,73.5,46.1,65.4,58.1,53.9,41.9,27.9,6.5,74.2,17.5
1,11489,70.2,15.8,9.6,25.4,61.5,84.0,16.0,67.8,5.0,71.4,48.9,75.2,65.6,51.1,34.6,25.1,8.4,84.5,23.3
2,13941,68.6,18.9,10.1,29.0,53.7,80.6,19.4,66.1,7.0,67.6,46.3,67.8,58.4,53.7,34.0,23.3,5.5,74.4,22.2
3,14109,68.0,14.6,13.5,28.1,57.4,75.9,24.1,65.5,7.9,71.6,43.8,73.1,61.2,56.3,38.1,23.3,4.8,84.4,14.0
4,15531,67.9,20.3,9.4,29.7,52.1,81.2,18.8,66.1,5.6,67.3,47.3,65.1,56.4,52.7,31.9,21.9,6.4,69.8,23.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,15067,54.5,16.8,5.5,22.3,41.6,86.8,13.2,54.4,3.3,56.2,45.0,58.7,55.2,55.0,20.5,13.3,6.3,62.5,12.5
198,10304,53.5,15.7,6.1,21.7,40.4,83.9,16.1,40.9,0.0,56.3,35.2,66.7,61.7,64.8,22.1,15.4,2.6,100.0,0.0
199,14176,53.4,14.6,9.8,24.4,43.1,75.4,24.6,46.5,11.6,62.5,38.5,69.4,53.1,61.5,19.6,13.8,5.5,78.4,30.8
200,14135,52.4,19.2,6.3,25.5,38.5,83.8,16.3,40.7,1.9,52.2,38.0,58.2,50.6,62.0,17.8,12.4,7.2,73.3,25.0


In [71]:
pitching_direction_2023 = recode_scraper('pitching','direction',2023)

In [72]:
pitching_direction_2023.columns = ['(P) ' + column for column in pitching_direction_2023.columns]
pitching_direction_2023['Pitcher Number'] = pitching_direction_2023['(P) Number']
pitching_direction_2023.columns

Index(['(P) Number', '(P) 이름', '(P) Position', '(P) Games, 출장',
       '(P) Innings Pitched, 이닝', '(P) Left ball%, 전체 좌측 타구 비율',
       '(P) Left center ball%, 전체 좌중앙 타구 비율', '(P) Center ball%, 전체 중앙 타구 비율',
       '(P) Right center ball%, 전체 우중앙 타구 비율', '(P) Right ball%, 전체 우측 타구 비율',
       '(P) Pull-side ball%, 전체 당겨친 타구 비율',
       '(P) Opposite-side ball%, 전체 밀어친 타구 비율', '(P) Left balls, 전체 좌측 타구 개수',
       '(P) Left center balls, 전체 좌중앙 타구 개수', '(P) Center balls, 전체 중앙 타구 개수',
       '(P) Right center balls, 전체 우중앙 타구 개수', '(P) Right balls, 전체 우측 타구 개수',
       '(P) Pull-side balls, 전체 당겨친 타구 개수',
       '(P) Opposite-side balls, 전체 밀어친 타구 개수', '(P) Left Hits, 전체 좌측 안타 개수',
       '(P) Left center Hits, 전체 좌중앙 안타 개수', '(P) Center Hits, 전체 중앙 안타 개수',
       '(P) Right center Hits, 전체 우중앙 안타 개수', '(P) Right Hits, 전체 우측 안타 개수',
       '(P) Pull-side Hits, 전체 당겨친 안타 개수',
       '(P) Opposite-side Hits, 전체 밀어친 안타 개수',
       '(P) Batting Average on Left balls, 좌측 타구 타율',
       '(P) 

In [73]:
pitching_direction_2023 = pitching_direction_2023[['Pitcher Number', '(P) Left ball%, 전체 좌측 타구 비율',
       '(P) Left center ball%, 전체 좌중앙 타구 비율', '(P) Center ball%, 전체 중앙 타구 비율',
       '(P) Right center ball%, 전체 우중앙 타구 비율', '(P) Right ball%, 전체 우측 타구 비율',
       '(P) Pull-side ball%, 전체 당겨친 타구 비율']]
pitching_direction_2023

,Pitcher Number,"(P) Left ball%, 전체 좌측 타구 비율","(P) Left center ball%, 전체 좌중앙 타구 비율","(P) Center ball%, 전체 중앙 타구 비율","(P) Right center ball%, 전체 우중앙 타구 비율","(P) Right ball%, 전체 우측 타구 비율","(P) Pull-side ball%, 전체 당겨친 타구 비율"
0,13003,17.5,27.5,17.5,17.5,20.0,57.6
1,11166,24.4,13.4,18.6,20.9,22.7,66.4
2,11415,16.7,16.7,20.4,19.8,26.5,52.3
3,15496,8.0,24.0,18.0,26.0,24.0,65.9
4,10909,22.3,24.8,15.7,17.8,19.4,57.8
...,...,...,...,...,...,...,...
197,11284,27.1,19.4,8.3,22.2,22.9,63.6
198,11144,19.6,28.3,23.9,17.4,10.9,62.9
199,14600,18.4,26.5,26.5,4.1,24.5,58.3
200,15435,20.0,25.7,14.3,25.7,14.3,53.3


In [74]:
pitching_player_data = pd.merge(pitching_balltype_2023,pitching_pitch_2023)
pitching_player_data = pd.merge(pitching_direction_2023,pitching_player_data)
pitching_player_data

,Pitcher Number,"(P) Left ball%, 전체 좌측 타구 비율","(P) Left center ball%, 전체 좌중앙 타구 비율","(P) Center ball%, 전체 중앙 타구 비율","(P) Right center ball%, 전체 우중앙 타구 비율","(P) Right ball%, 전체 우측 타구 비율","(P) Pull-side ball%, 전체 당겨친 타구 비율","2-Seamer Fastball Velocity, 평균구속 (투심)","4-Seamer Fastball Velocity, 평균구속 (포심)","Cutter Velocity, 평균구속 (커터)",...,(P) 투스트라이크 카운트 투구 대비 삼진 결정 비율,"(P) Strike Zone%, 존 안에 들어온 투구 비율","(P) Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율","(P) Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율","(P) Out Zone%, 존 밖에 들어온 투구 비율","(P) Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율","(P) Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율","(P) Meatball Zone%, 존 한가운데 들어온 투구 비율","(P) Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율","(P) Looking Strike Out%, 루킹 삼진 비율"
0,13003,17.5,27.5,17.5,17.5,20.0,57.6,143.3,143.5,137.5,...,59.7,38.1,64.8,56.0,61.9,29.1,20.9,2.9,42.9,18.2
1,11166,24.4,13.4,18.6,20.9,22.7,66.4,NaN,142.8,NaN,...,69.7,43.0,72.8,59.2,57.0,36.0,22.7,5.5,75.4,18.8
2,11415,16.7,16.7,20.4,19.8,26.5,52.3,NaN,145.1,NaN,...,67.5,47.3,71.0,59.9,52.7,28.1,19.5,5.9,84.9,20.0
3,15496,8.0,24.0,18.0,26.0,24.0,65.9,NaN,140.6,NaN,...,68.5,43.8,65.5,58.8,56.3,21.6,14.4,5.5,73.3,27.3
4,10909,22.3,24.8,15.7,17.8,19.4,57.8,NaN,137.2,NaN,...,67.0,39.3,65.7,57.8,60.7,36.8,23.4,4.3,81.5,10.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,11284,27.1,19.4,8.3,22.2,22.9,63.6,136.3,138.6,NaN,...,72.7,42.4,73.8,69.4,57.6,36.8,27.3,6.5,81.6,23.5
198,11144,19.6,28.3,23.9,17.4,10.9,62.9,NaN,143.8,NaN,...,75.7,47.1,61.3,50.8,52.9,30.9,23.7,6.5,88.2,50.0
199,14600,18.4,26.5,26.5,4.1,24.5,58.3,139.5,139.8,139.3,...,58.8,43.0,70.7,65.7,57.0,26.7,22.1,8.3,68.4,0.0
200,15435,20.0,25.7,14.3,25.7,14.3,53.3,144.7,143.8,NaN,...,60.0,40.4,51.6,45.2,59.6,19.0,9.5,4.8,54.5,21.4


In [75]:
pitching_split_data = []

for id in list(pitching_player_data['Pitcher Number']):
    pitching_split_num = split_finder(id,'pitching')
    pitching_split_data.append(pitching_split_num)

pitching_player_data['(P) 투타'] = pitching_split_data

13003
11166
11415
15496
10909
14106
12898
11110
11094
13009
11355
13130
15127
10220
15057
12885
13088
10367
14143
13061
12934
14669
14708
11353
15146
15653
12637
11070
13071
10690
14763
11303
13242
15153
10217
14108
13942
10533
15455
15646
13941
14155
15531
10535
11323
10437
10427
15486
14849
11242
15535
14798
11318
13085
15432
14617
10652
11489
10131
12989
13122
13091
13147
12537
14869
14145
12295
15533
10609
10363
10784
15063
14589
14788
15143
14113
14769
10126
12847
15003
12568
14156
15013
14595
15462
11310
12858
15071
10203
14784
13114
14871
14176
10685
15089
15536
15528
10453
13934
15534
14770
11379
11308
15073
14581
14134
15643
14608
10527
10124
14792
14627
13148
15644
15099
15147
10156
11411
10058
12852
10871
10749
12605
14135
13244
11394
14624
10825
12850
15474
12944
10610
15011
12565
11300
14111
10452
10075
10706
12930
10863
11311
12871
14588
14808
14621
14780
15067
11168
15461
12572
11291
13115
10516
13156
14615
10696
10304
11161
13080
11164
13126
10783
10523
11317
15119
1290

In [76]:
pitching_player_data.to_excel('pitching_player_data.xlsx')

In [77]:
pitching_player_data

,Pitcher Number,"(P) Left ball%, 전체 좌측 타구 비율","(P) Left center ball%, 전체 좌중앙 타구 비율","(P) Center ball%, 전체 중앙 타구 비율","(P) Right center ball%, 전체 우중앙 타구 비율","(P) Right ball%, 전체 우측 타구 비율","(P) Pull-side ball%, 전체 당겨친 타구 비율","2-Seamer Fastball Velocity, 평균구속 (투심)","4-Seamer Fastball Velocity, 평균구속 (포심)","Cutter Velocity, 평균구속 (커터)",...,"(P) Strike Zone%, 존 안에 들어온 투구 비율","(P) Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율","(P) Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율","(P) Out Zone%, 존 밖에 들어온 투구 비율","(P) Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율","(P) Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율","(P) Meatball Zone%, 존 한가운데 들어온 투구 비율","(P) Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율","(P) Looking Strike Out%, 루킹 삼진 비율",(P) 투타
0,13003,17.5,27.5,17.5,17.5,20.0,57.6,143.3,143.5,137.5,...,38.1,64.8,56.0,61.9,29.1,20.9,2.9,42.9,18.2,0.00
1,11166,24.4,13.4,18.6,20.9,22.7,66.4,NaN,142.8,NaN,...,43.0,72.8,59.2,57.0,36.0,22.7,5.5,75.4,18.8,0.00
2,11415,16.7,16.7,20.4,19.8,26.5,52.3,NaN,145.1,NaN,...,47.3,71.0,59.9,52.7,28.1,19.5,5.9,84.9,20.0,0.00
3,15496,8.0,24.0,18.0,26.0,24.0,65.9,NaN,140.6,NaN,...,43.8,65.5,58.8,56.3,21.6,14.4,5.5,73.3,27.3,0.00
4,10909,22.3,24.8,15.7,17.8,19.4,57.8,NaN,137.2,NaN,...,39.3,65.7,57.8,60.7,36.8,23.4,4.3,81.5,10.5,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,11284,27.1,19.4,8.3,22.2,22.9,63.6,136.3,138.6,NaN,...,42.4,73.8,69.4,57.6,36.8,27.3,6.5,81.6,23.5,0.00
198,11144,19.6,28.3,23.9,17.4,10.9,62.9,NaN,143.8,NaN,...,47.1,61.3,50.8,52.9,30.9,23.7,6.5,88.2,50.0,1.00
199,14600,18.4,26.5,26.5,4.1,24.5,58.3,139.5,139.8,139.3,...,43.0,70.7,65.7,57.0,26.7,22.1,8.3,68.4,0.0,0.00
200,15435,20.0,25.7,14.3,25.7,14.3,53.3,144.7,143.8,NaN,...,40.4,51.6,45.2,59.6,19.0,9.5,4.8,54.5,21.4,1.00


In [78]:
pitching_player_data.columns

Index(['Pitcher Number', '(P) Left ball%, 전체 좌측 타구 비율',
       '(P) Left center ball%, 전체 좌중앙 타구 비율', '(P) Center ball%, 전체 중앙 타구 비율',
       '(P) Right center ball%, 전체 우중앙 타구 비율', '(P) Right ball%, 전체 우측 타구 비율',
       '(P) Pull-side ball%, 전체 당겨친 타구 비율',
       '2-Seamer Fastball Velocity, 평균구속 (투심)',
       '4-Seamer Fastball Velocity, 평균구속 (포심)', 'Cutter Velocity, 평균구속 (커터)',
       'Curve Velocity, 평균구속 (커브)', 'Slider Velocity, 평균구속 (슬라이더)',
       'Changeup Velocity, 평균구속 (체인지업)', 'Sinker Velocity, 평균구속 (싱커)',
       'Forkball Velocity, 평균구속 (포크볼)',
       '2-Seamer Fastball Pitch Value per 100, 구종가치/100 (투심)',
       '4-Seamer Fastball Pitch Value per 100, 구종가치/100 (포심)',
       'Cutter Pitch Value per 100, 구종가치/100 (커터)',
       'Curve Pitch Value per 100, 구종가치/100 (커브)',
       'Slider Pitch Value per 100, 구종가치/100 (슬라이더)',
       'Changeup Pitch Value per 100, 구종가치/100 (체인지업)',
       'Sinker Pitch Value per 100, 구종가치/100 (싱커)',
       'Forkball Pitch Value per 100, 구종가치/100

In [79]:
batting_player_data.columns

Index(['Hitter Number', '구종가치/100 (투심)', '구종가치/100 (포심)', '구종가치/100 (커터)',
       '구종가치/100 (커브)', '구종가치/100 (슬라이더)', '구종가치/100 (체인지업)', '구종가치/100 (싱커)',
       '구종가치/100 (포크볼)', 'Strike%, 전체 투구 대비 스트라이크',
       'Called Strike%, 전체 투구 대비 루킹 스트라이크%', 'Whiff%, 전체 투구 대비 헛스윙 스트라이크%',
       'CSW%, 전체 투구 대비 루킹+헛스윙 스트라이크%', 'Swing%, 스윙 비율', '스윙 대비 콘택트 비율',
       '스윙 대비 헛스윙 비율', '초구 스트라이크 비율', '초구 스윙 비율', '투스트라이크 카운트 투구 대비 삼진 결정 비율',
       'Strike Zone%, 존 안에 들어온 투구 비율',
       'Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율',
       'Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율',
       'Out Zone%, 존 밖에 들어온 투구 비율', 'Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율',
       'Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율',
       'Meatball Zone%, 존 한가운데 들어온 투구 비율',
       'Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율',
       'Shadow Zone%, 쉐도우존에 들어온 투구 비율', 'Looking Strike Out%, 루킹 삼진 비율',
       'Batting Average on Balls In Play, 인플레이 타구의 안타 비율', '땅볼%', '내야 뜬볼%',
       '외야 뜬볼%', '뜬볼%', '라인드라이브%', '홈런 / 뜬볼%', '내야안타%

In [80]:
# 2024년 4월 24일 현재 전체 시즌 파크 팩터
park_factors = {'잠실':895, '사직':1064,'창원':1051,'대구':1111,'수원':1020,'문학':985, '고척':965, '대전':1019, '광주':1000,
               '울산':1000,'포항':1000}

park_factors_list = []
for stadium in list(gamelog_2023['경기장']):
    park_factors_list.append(park_factors[stadium])
    
gamelog_2023['파크팩터'] = park_factors_list

In [81]:
gamelog_2023.columns

Index(['날짜', '경기장', 'Pitcher Number', 'Hitter Number', '회', '내야 수비 RAA',
       '외야 수비 RAA', '포수 수비 RAA', '투수 이름', '타자 이름', '투구수', '결과', '이전상황', '이후상황',
       'LEV', 'REs', 'REa', 'WPe', 'WPa', '파크팩터'],
      dtype='object')

In [105]:
gamelog_agg_2023 = pd.merge(gamelog_2023, batting_player_data)

In [106]:
gamelog_agg_2023 = pd.merge(gamelog_agg_2023,pitching_player_data)

In [107]:
gamelog_agg_2023

,날짜,경기장,Pitcher Number,Hitter Number,회,내야 수비 RAA,외야 수비 RAA,포수 수비 RAA,투수 이름,타자 이름,...,"(P) Strike Zone%, 존 안에 들어온 투구 비율","(P) Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율","(P) Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율","(P) Out Zone%, 존 밖에 들어온 투구 비율","(P) Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율","(P) Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율","(P) Meatball Zone%, 존 한가운데 들어온 투구 비율","(P) Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율","(P) Looking Strike Out%, 루킹 삼진 비율",(P) 투타
0,04-01,잠실,13941,14571,1회초,19.37,-0.87,2.996907,알칸타라,안권수,...,46.3,67.8,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0
1,04-01,잠실,13941,14571,2회초,19.37,-0.87,2.996907,알칸타라,안권수,...,46.3,67.8,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0
2,04-01,잠실,13941,14571,4회초,19.37,-0.87,2.996907,알칸타라,안권수,...,46.3,67.8,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0
3,05-09,사직,13941,14571,1회말,12.45,-1.33,2.996907,알칸타라,안권수,...,46.3,67.8,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0
4,05-09,사직,13941,14571,3회말,12.45,-1.33,2.996907,알칸타라,안권수,...,46.3,67.8,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56899,05-10,수원,14600,14220,4회초,-8.48,7.96,-14.196000,소형준,서호철,...,43.0,70.7,65.7,57.0,26.7,22.1,8.3,68.4,0.0,0.0
56900,05-03,문학,14600,13195,2회말,4.60,5.06,1.826552,소형준,최상민,...,43.0,70.7,65.7,57.0,26.7,22.1,8.3,68.4,0.0,0.0
56901,05-03,문학,14600,13195,4회말,4.60,5.06,1.826552,소형준,최상민,...,43.0,70.7,65.7,57.0,26.7,22.1,8.3,68.4,0.0,0.0
56902,05-10,수원,14600,11172,1회초,-8.48,7.96,-14.196000,소형준,권희동,...,43.0,70.7,65.7,57.0,26.7,22.1,8.3,68.4,0.0,0.0


In [108]:
gamelog_agg_2023.columns

Index(['날짜', '경기장', 'Pitcher Number', 'Hitter Number', '회', '내야 수비 RAA',
       '외야 수비 RAA', '포수 수비 RAA', '투수 이름', '타자 이름',
       ...
       '(P) Strike Zone%, 존 안에 들어온 투구 비율',
       '(P) Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율',
       '(P) Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율',
       '(P) Out Zone%, 존 밖에 들어온 투구 비율',
       '(P) Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율',
       '(P) Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율',
       '(P) Meatball Zone%, 존 한가운데 들어온 투구 비율',
       '(P) Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율',
       '(P) Looking Strike Out%, 루킹 삼진 비율', '(P) 투타'],
      dtype='object', length=104)

In [109]:
gamelog_agg_2023 = gamelog_agg_2023[['날짜','Pitcher Number', 'Hitter Number','내야 수비 RAA',
       '외야 수비 RAA', '포수 수비 RAA', '구종가치/100 (투심)',
       '구종가치/100 (포심)', '구종가치/100 (커터)', '구종가치/100 (커브)', '구종가치/100 (슬라이더)',
       '구종가치/100 (체인지업)', '구종가치/100 (싱커)', '구종가치/100 (포크볼)',
       'Strike%, 전체 투구 대비 스트라이크', 'Called Strike%, 전체 투구 대비 루킹 스트라이크%',
       'Whiff%, 전체 투구 대비 헛스윙 스트라이크%', 'CSW%, 전체 투구 대비 루킹+헛스윙 스트라이크%',
       'Swing%, 스윙 비율', '스윙 대비 콘택트 비율', '스윙 대비 헛스윙 비율', '초구 스트라이크 비율',
       '초구 스윙 비율', '투스트라이크 카운트 투구 대비 삼진 결정 비율', 'Strike Zone%, 존 안에 들어온 투구 비율',
       'Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율',
       'Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율',
       'Out Zone%, 존 밖에 들어온 투구 비율', 'Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율',
       'Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율',
       'Meatball Zone%, 존 한가운데 들어온 투구 비율',
       'Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율',
       'Shadow Zone%, 쉐도우존에 들어온 투구 비율', 'Looking Strike Out%, 루킹 삼진 비율',
       'Batting Average on Balls In Play, 인플레이 타구의 안타 비율', '땅볼%', '내야 뜬볼%',
       '외야 뜬볼%', '뜬볼%', '라인드라이브%', '홈런 / 뜬볼%', '내야안타%', '전체 좌측 타구 비율',
       '전체 좌중앙 타구 비율', '전체 중앙 타구 비율', '전체 우중앙 타구 비율', '전체 우측 타구 비율', '투타',
       '(P) Left ball%, 전체 좌측 타구 비율', '(P) Left center ball%, 전체 좌중앙 타구 비율',
       '(P) Center ball%, 전체 중앙 타구 비율', '(P) Right center ball%, 전체 우중앙 타구 비율',
       '(P) Right ball%, 전체 우측 타구 비율', '(P) Pull-side ball%, 전체 당겨친 타구 비율',
        '2-Seamer Fastball Velocity, 평균구속 (투심)',
       '4-Seamer Fastball Velocity, 평균구속 (포심)', 'Cutter Velocity, 평균구속 (커터)',
       'Curve Velocity, 평균구속 (커브)', 'Slider Velocity, 평균구속 (슬라이더)',
       'Changeup Velocity, 평균구속 (체인지업)', 'Sinker Velocity, 평균구속 (싱커)',
       'Forkball Velocity, 평균구속 (포크볼)',
       '2-Seamer Fastball Pitch Value per 100, 구종가치/100 (투심)',
       '4-Seamer Fastball Pitch Value per 100, 구종가치/100 (포심)',
       'Cutter Pitch Value per 100, 구종가치/100 (커터)',
       'Curve Pitch Value per 100, 구종가치/100 (커브)',
       'Slider Pitch Value per 100, 구종가치/100 (슬라이더)',
       'Changeup Pitch Value per 100, 구종가치/100 (체인지업)',
       'Sinker Pitch Value per 100, 구종가치/100 (싱커)',
       'Forkball Pitch Value per 100, 구종가치/100 (포크볼)',
       '(P) Strike%, 전체 투구 대비 스트라이크', '(P) Called Strike%, 전체 투구 대비 루킹 스트라이크%',
       '(P) Whiff%, 전체 투구 대비 헛스윙 스트라이크%', '(P) CSW%, 전체 투구 대비 루킹+헛스윙 스트라이크%',
       '(P) Swing%, 스윙 비율', '(P) 스윙 대비 콘택트 비율', '(P) 스윙 대비 헛스윙 비율',
       '(P) 초구 스트라이크 비율', '(P) 초구 스윙 비율', '(P) 투스트라이크 카운트 투구 대비 삼진 결정 비율',
       '(P) Strike Zone%, 존 안에 들어온 투구 비율',
       '(P) Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율',
       '(P) Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율',
       '(P) Out Zone%, 존 밖에 들어온 투구 비율',
       '(P) Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율',
       '(P) Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율',
       '(P) Meatball Zone%, 존 한가운데 들어온 투구 비율',
       '(P) Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율',
       '(P) Looking Strike Out%, 루킹 삼진 비율', '(P) 투타', '파크팩터', '결과']]
gamelog_agg_2023

,날짜,Pitcher Number,Hitter Number,내야 수비 RAA,외야 수비 RAA,포수 수비 RAA,구종가치/100 (투심),구종가치/100 (포심),구종가치/100 (커터),구종가치/100 (커브),...,"(P) Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율","(P) Out Zone%, 존 밖에 들어온 투구 비율","(P) Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율","(P) Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율","(P) Meatball Zone%, 존 한가운데 들어온 투구 비율","(P) Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율","(P) Looking Strike Out%, 루킹 삼진 비율",(P) 투타,파크팩터,결과
0,04-01,13941,14571,19.37,-0.87,2.996907,0.945946,-0.690789,-11.000000,-2.666667,...,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0,895,헛스윙 삼진
1,04-01,13941,14571,19.37,-0.87,2.996907,0.945946,-0.690789,-11.000000,-2.666667,...,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0,895,좌익수 파울플라이 아웃
2,04-01,13941,14571,19.37,-0.87,2.996907,0.945946,-0.690789,-11.000000,-2.666667,...,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0,895,좌익수 방면 1루타
3,05-09,13941,14571,12.45,-1.33,2.996907,0.945946,-0.690789,-11.000000,-2.666667,...,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0,1064,좌익수 방면 1루타
4,05-09,13941,14571,12.45,-1.33,2.996907,0.945946,-0.690789,-11.000000,-2.666667,...,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0,1064,유격수 땅볼 아웃
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56899,05-10,14600,14220,-8.48,7.96,-14.196000,5.061728,-2.184250,3.888889,-0.890052,...,65.7,57.0,26.7,22.1,8.3,68.4,0.0,0.0,1020,중견수 방면 1루타
56900,05-03,14600,13195,4.60,5.06,1.826552,-10.909091,-1.578947,NaN,NaN,...,65.7,57.0,26.7,22.1,8.3,68.4,0.0,0.0,985,2루수 땅볼 아웃
56901,05-03,14600,13195,4.60,5.06,1.826552,-10.909091,-1.578947,NaN,NaN,...,65.7,57.0,26.7,22.1,8.3,68.4,0.0,0.0,985,좌익수 파울플라이 아웃
56902,05-10,14600,11172,-8.48,7.96,-14.196000,-1.111111,0.124611,-2.830189,-0.543478,...,65.7,57.0,26.7,22.1,8.3,68.4,0.0,0.0,1020,선행주자아웃 출루


In [110]:
for i in set(list(gamelog_agg_2023['결과'])):
    print(i)

1루주자 김태연 : 2루 도루
1루주자 강한울 : 포일로 2루 진루
1루주자 김민성 : 2루 도루
1루주자 박찬호 : 포일로 2루 진루
1루주자 에레디아 : 폭투로 2루 진루
2루주자 박찬호 : 폭투로 3루 진루
1루주자 김민성 : 2루 이중도루
1루주자 손호영 : 2루 도루
1루주자 오재일 : 폭투로 2루 진루
1루주자 양찬열 : 폭투로 2루 진루
2루주자 박동원 : 폭투로 3루 진루
1루주자 박건우 : 2루도루실패
1루주자 변우혁 : 보크로 2루 진루
3루주자 김도영 : 폭투로 홈인
 1루수 방면 2루타
1루주자 허경민 : 2루도루실패
1루주자 김민석 : 이중도루 실패 사이 2루 진루
1루주자 이용규 : 폭투사이 진루실패 2루 아웃
1루주자 김선빈 : 2루도루실패
1루주자 피렐라 : 투수 1루 견제아웃
 타구맞음 아웃
1루주자 김재환 : 2루도루실패
1루주자 최준우 : 2루도루실패
1루주자 구자욱 : 2루도루실패
2루주자 강진성 : 폭투로 3루 진루
 유격수 직선타 아웃
1루주자 손아섭 : 폭투로 2루 진루
1루주자 알포드 : 2루 이중도루
1루주자 박석민 : 폭투로 2루 진루
1루주자 김한별 : 폭투로 2루 진루
 2루수 희생번트 아웃
1루주자 박찬호 : 2루도루실패
1루주자 김도영 : 투수 1루 견제아웃
2루주자 박민우 : 3루 도루
1루주자 강승호 : 2루도루실패
2루주자 이형종 : 포일로 3루 진루
1루주자 박민우 : 2루 도루
 2루수 땅볼 출루
1루주자 유로결 : 폭투로 2루 진루
1루주자 박민우 : 2루도루실패
1루주자 박성한 : 폭투로 2루 진루
2루주자 강승호 : 3루도루실패
 1루수 포구실책 출루
1루주자 윤동희 : 포일로 2루 진루
 1루수 희생번트 포구실책 출루
3루주자 최지훈 : 폭투로 홈인
1루주자 서호철 : 포일로 2루 진루
 3루수 플라이 아웃
1루주자 이성규 : 2루도루실패
2루주자 이도윤 : 폭투로 3루 진루
타자주자  1루 진루
1루주자 오지환 : 투수 1루 견제아웃
1루주자 박준영 : 2루 이중도루
2루주자 조수행 : 3

가정
1. 보크, 폭투, 도루, 포일, 타구 맞음 같은 돌발 상황 X
2. 트리플 플레이 X
3. 타석에서의 모든 경우의 수는 1루타(1베이스 출루), 2루타(2베이스 출루), 3루타(3베이스 출루), 홈런, 아웃(삼진 + 플라이 아웃), 더블플레이, 사사구(볼넷, 데드볼)가 있다.
4. 아웃 시 주자는 진루하지 않는다.
5. 에러는 1베이스 진루로 가정한다.
6. 야수선택 출루는 페어 지역에 떨어진 땅볼을 잡은 내야수가 정상적으로 1루로 송구하였으면 타자 주자가 충분히 아웃될 수 있는데 다른 주자를 잡기 위해 다른 베이스로 송구했을 때 타자에게 주어지는 기록이므로 원래는 타자가 아웃인 타구라고 가정해 아웃으로 판정한다.
7. 땅볼 출루의 경우 상대 수비수의 에러나 선행 주자가 아웃되어서 타자 주자가 살아남은 경우 주어지는 기록이므로 원래는 타자가 아웃인 타구라고 가정해 아웃으로 판정한다.
8. 희생번트는 주자가 있을때만 나오는 특수한 상황이므로 제외한다.

In [111]:
result_list = []

for result in list(gamelog_agg_2023['결과']):
    if '플라이 아웃' in result:
        result_list.append('아웃')
    elif '땅볼 아웃' in result:
        result_list.append('아웃')
    elif '병살타' in result:
        result_list.append('더블플레이')
    elif '실책' in result:
        result_list.append('1루타')
    elif '1루타' in result:
        result_list.append('1루타')
    elif '2루타' in result:
        result_list.append('2루타')
    elif '3루타' in result:
        result_list.append('3루타')
    elif '내야 안타' in result:
        result_list.append('1루타')
    elif '홈런' in result:
        result_list.append('홈런')
    elif '직선타 아웃' in result:
        result_list.append('아웃')
    elif '삼진' in result:
        result_list.append('아웃')
    elif '야수선택' in result:
        result_list.append('아웃')
    elif '땅볼 출루' in result:
        result_list.append('아웃')
    elif '사구' in result:
        result_list.append('사사구')
    elif '3번트 아웃' in result:
        result_list.append('아웃')
    elif '4구' in result:
        result_list.append('사사구')
    elif '선행주자아웃 출루' in result:
        result_list.append('아웃')
    else:
        result_list.append(None)

gamelog_agg_2023['Result'] = result_list

C:\Users\kimhw\AppData\Local\Temp\ipykernel_28724\3147862612.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gamelog_agg_2023['Result'] = result_list


In [112]:
gamelog_agg_2023['Result']

0         아웃
1         아웃
2        1루타
3        1루타
4         아웃
        ... 
56899    1루타
56900     아웃
56901     아웃
56902     아웃
56903    1루타
Name: Result, Length: 56904, dtype: object

In [113]:
gamelog_agg_2023 = gamelog_agg_2023[['날짜','Pitcher Number', 'Hitter Number','내야 수비 RAA',
       '외야 수비 RAA', '포수 수비 RAA', '구종가치/100 (투심)',
       '구종가치/100 (포심)', '구종가치/100 (커터)', '구종가치/100 (커브)', '구종가치/100 (슬라이더)',
       '구종가치/100 (체인지업)', '구종가치/100 (싱커)', '구종가치/100 (포크볼)',
       'Strike%, 전체 투구 대비 스트라이크', 'Called Strike%, 전체 투구 대비 루킹 스트라이크%',
       'Whiff%, 전체 투구 대비 헛스윙 스트라이크%', 'CSW%, 전체 투구 대비 루킹+헛스윙 스트라이크%',
       'Swing%, 스윙 비율', '스윙 대비 콘택트 비율', '스윙 대비 헛스윙 비율', '초구 스트라이크 비율',
       '초구 스윙 비율', '투스트라이크 카운트 투구 대비 삼진 결정 비율', 'Strike Zone%, 존 안에 들어온 투구 비율',
       'Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율',
       'Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율',
       'Out Zone%, 존 밖에 들어온 투구 비율', 'Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율',
       'Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율',
       'Meatball Zone%, 존 한가운데 들어온 투구 비율',
       'Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율',
       'Shadow Zone%, 쉐도우존에 들어온 투구 비율', 'Looking Strike Out%, 루킹 삼진 비율',
       'Batting Average on Balls In Play, 인플레이 타구의 안타 비율', '땅볼%', '내야 뜬볼%',
       '외야 뜬볼%', '뜬볼%', '라인드라이브%', '홈런 / 뜬볼%', '내야안타%', '전체 좌측 타구 비율',
       '전체 좌중앙 타구 비율', '전체 중앙 타구 비율', '전체 우중앙 타구 비율', '전체 우측 타구 비율', '투타',
       '(P) Left ball%, 전체 좌측 타구 비율', '(P) Left center ball%, 전체 좌중앙 타구 비율',
       '(P) Center ball%, 전체 중앙 타구 비율', '(P) Right center ball%, 전체 우중앙 타구 비율',
       '(P) Right ball%, 전체 우측 타구 비율', '(P) Pull-side ball%, 전체 당겨친 타구 비율',
        '2-Seamer Fastball Velocity, 평균구속 (투심)',
       '4-Seamer Fastball Velocity, 평균구속 (포심)', 'Cutter Velocity, 평균구속 (커터)',
       'Curve Velocity, 평균구속 (커브)', 'Slider Velocity, 평균구속 (슬라이더)',
       'Changeup Velocity, 평균구속 (체인지업)', 'Sinker Velocity, 평균구속 (싱커)',
       'Forkball Velocity, 평균구속 (포크볼)',
       '2-Seamer Fastball Pitch Value per 100, 구종가치/100 (투심)',
       '4-Seamer Fastball Pitch Value per 100, 구종가치/100 (포심)',
       'Cutter Pitch Value per 100, 구종가치/100 (커터)',
       'Curve Pitch Value per 100, 구종가치/100 (커브)',
       'Slider Pitch Value per 100, 구종가치/100 (슬라이더)',
       'Changeup Pitch Value per 100, 구종가치/100 (체인지업)',
       'Sinker Pitch Value per 100, 구종가치/100 (싱커)',
       'Forkball Pitch Value per 100, 구종가치/100 (포크볼)',
       '(P) Strike%, 전체 투구 대비 스트라이크', '(P) Called Strike%, 전체 투구 대비 루킹 스트라이크%',
       '(P) Whiff%, 전체 투구 대비 헛스윙 스트라이크%', '(P) CSW%, 전체 투구 대비 루킹+헛스윙 스트라이크%',
       '(P) Swing%, 스윙 비율', '(P) 스윙 대비 콘택트 비율', '(P) 스윙 대비 헛스윙 비율',
       '(P) 초구 스트라이크 비율', '(P) 초구 스윙 비율', '(P) 투스트라이크 카운트 투구 대비 삼진 결정 비율',
       '(P) Strike Zone%, 존 안에 들어온 투구 비율',
       '(P) Strike Zone Swing%, 존 안에 들어온 투구 대비 스윙 비율',
       '(P) Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율',
       '(P) Out Zone%, 존 밖에 들어온 투구 비율',
       '(P) Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율',
       '(P) Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율',
       '(P) Meatball Zone%, 존 한가운데 들어온 투구 비율',
       '(P) Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율',
       '(P) Looking Strike Out%, 루킹 삼진 비율', '(P) 투타', '파크팩터', 'Result']]
gamelog_agg_2023

,날짜,Pitcher Number,Hitter Number,내야 수비 RAA,외야 수비 RAA,포수 수비 RAA,구종가치/100 (투심),구종가치/100 (포심),구종가치/100 (커터),구종가치/100 (커브),...,"(P) Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율","(P) Out Zone%, 존 밖에 들어온 투구 비율","(P) Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율","(P) Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율","(P) Meatball Zone%, 존 한가운데 들어온 투구 비율","(P) Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율","(P) Looking Strike Out%, 루킹 삼진 비율",(P) 투타,파크팩터,Result
0,04-01,13941,14571,19.37,-0.87,2.996907,0.945946,-0.690789,-11.000000,-2.666667,...,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0,895,아웃
1,04-01,13941,14571,19.37,-0.87,2.996907,0.945946,-0.690789,-11.000000,-2.666667,...,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0,895,아웃
2,04-01,13941,14571,19.37,-0.87,2.996907,0.945946,-0.690789,-11.000000,-2.666667,...,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0,895,1루타
3,05-09,13941,14571,12.45,-1.33,2.996907,0.945946,-0.690789,-11.000000,-2.666667,...,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0,1064,1루타
4,05-09,13941,14571,12.45,-1.33,2.996907,0.945946,-0.690789,-11.000000,-2.666667,...,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0,1064,아웃
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56899,05-10,14600,14220,-8.48,7.96,-14.196000,5.061728,-2.184250,3.888889,-0.890052,...,65.7,57.0,26.7,22.1,8.3,68.4,0.0,0.0,1020,1루타
56900,05-03,14600,13195,4.60,5.06,1.826552,-10.909091,-1.578947,NaN,NaN,...,65.7,57.0,26.7,22.1,8.3,68.4,0.0,0.0,985,아웃
56901,05-03,14600,13195,4.60,5.06,1.826552,-10.909091,-1.578947,NaN,NaN,...,65.7,57.0,26.7,22.1,8.3,68.4,0.0,0.0,985,아웃
56902,05-10,14600,11172,-8.48,7.96,-14.196000,-1.111111,0.124611,-2.830189,-0.543478,...,65.7,57.0,26.7,22.1,8.3,68.4,0.0,0.0,1020,아웃


In [114]:
# 1루타(1베이스 출루), 2루타(2베이스 출루), 3루타(3베이스 출루), 홈런, 아웃(삼진 + 플라이 아웃), 더블플레이, 사사구 외 상황 제거
gamelog_agg_2023 = gamelog_agg_2023.dropna(subset=['Result'])
gamelog_agg_2023

,날짜,Pitcher Number,Hitter Number,내야 수비 RAA,외야 수비 RAA,포수 수비 RAA,구종가치/100 (투심),구종가치/100 (포심),구종가치/100 (커터),구종가치/100 (커브),...,"(P) Strike Zone Contact%, 존 안에 들어온 투구 대비 콘택트 비율","(P) Out Zone%, 존 밖에 들어온 투구 비율","(P) Out Zone Swing%, 존 밖에 들어온 투구 대비 스윙 비율","(P) Out Zone Contact%, 존 밖에 들어온 투구 대비 콘택트 비율","(P) Meatball Zone%, 존 한가운데 들어온 투구 비율","(P) Meatball Swing%, 존 한가운데 들어온 투구 대비 스윙 비율","(P) Looking Strike Out%, 루킹 삼진 비율",(P) 투타,파크팩터,Result
0,04-01,13941,14571,19.37,-0.87,2.996907,0.945946,-0.690789,-11.000000,-2.666667,...,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0,895,아웃
1,04-01,13941,14571,19.37,-0.87,2.996907,0.945946,-0.690789,-11.000000,-2.666667,...,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0,895,아웃
2,04-01,13941,14571,19.37,-0.87,2.996907,0.945946,-0.690789,-11.000000,-2.666667,...,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0,895,1루타
3,05-09,13941,14571,12.45,-1.33,2.996907,0.945946,-0.690789,-11.000000,-2.666667,...,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0,1064,1루타
4,05-09,13941,14571,12.45,-1.33,2.996907,0.945946,-0.690789,-11.000000,-2.666667,...,58.4,53.7,34.0,23.3,5.5,74.4,22.2,0.0,1064,아웃
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56899,05-10,14600,14220,-8.48,7.96,-14.196000,5.061728,-2.184250,3.888889,-0.890052,...,65.7,57.0,26.7,22.1,8.3,68.4,0.0,0.0,1020,1루타
56900,05-03,14600,13195,4.60,5.06,1.826552,-10.909091,-1.578947,NaN,NaN,...,65.7,57.0,26.7,22.1,8.3,68.4,0.0,0.0,985,아웃
56901,05-03,14600,13195,4.60,5.06,1.826552,-10.909091,-1.578947,NaN,NaN,...,65.7,57.0,26.7,22.1,8.3,68.4,0.0,0.0,985,아웃
56902,05-10,14600,11172,-8.48,7.96,-14.196000,-1.111111,0.124611,-2.830189,-0.543478,...,65.7,57.0,26.7,22.1,8.3,68.4,0.0,0.0,1020,아웃


In [115]:
gamelog_agg_2023.to_excel('gamelog_agg_2023.xlsx')

In [116]:
gamelog_agg_2023.describe()

,내야 수비 RAA,외야 수비 RAA,포수 수비 RAA,구종가치/100 (투심),구종가치/100 (포심),구종가치/100 (커터),구종가치/100 (커브),구종가치/100 (슬라이더),구종가치/100 (체인지업),구종가치/100 (싱커),...,"2-Seamer Fastball Pitch Value per 100, 구종가치/100 (투심)","4-Seamer Fastball Pitch Value per 100, 구종가치/100 (포심)","Cutter Pitch Value per 100, 구종가치/100 (커터)","Curve Pitch Value per 100, 구종가치/100 (커브)","Slider Pitch Value per 100, 구종가치/100 (슬라이더)","Changeup Pitch Value per 100, 구종가치/100 (체인지업)","Sinker Pitch Value per 100, 구종가치/100 (싱커)","Forkball Pitch Value per 100, 구종가치/100 (포크볼)",(P) 투타,파크팩터
count,54030.000000,54030.000000,54030.000000,52367.000000,54030.000000,45304.000000,53654.000000,54030.000000,53875.000000,47361.000000,...,14656.000000,51692.000000,9938.000000,42806.000000,51443.000000,36027.000000,3112.000000,20231.000000,54030.000000,54030.000000
mean,6.440616,2.539528,0.121664,-0.046080,-0.535849,-1.555130,-0.202830,-0.901206,-0.479684,0.672408,...,-0.595818,0.588868,0.900509,-0.038102,0.833539,0.284032,0.424201,0.381095,0.306330,999.318527
std,15.645164,16.068376,4.789231,3.201347,1.339297,3.670384,1.972503,1.505257,1.975749,4.424741,...,2.440651,1.525423,2.517167,2.325643,2.034000,1.901928,1.784122,3.078004,0.438439,64.048661
min,-37.840000,-33.820000,-14.360000,-13.750000,-7.254902,-11.000000,-10.000000,-8.305085,-12.666667,-10.000000,...,-11.052632,-10.975610,-5.384615,-16.153846,-7.019231,-9.090909,-1.750000,-12.307692,0.000000,895.000000
25%,-4.150000,-9.370000,-1.556000,-2.115385,-1.440887,-3.823529,-1.237624,-1.818182,-1.561338,-2.352941,...,-1.200000,0.088737,0.000000,-0.873786,-0.043573,-0.569106,-1.072089,-0.115274,0.000000,965.000000
50%,8.900000,0.000000,1.826552,0.086957,-0.416667,-1.590909,-0.382166,-0.788913,-0.380435,0.158730,...,-0.096386,0.718954,1.052632,0.083333,0.937500,0.541872,0.303030,0.666667,0.000000,1000.000000
75%,18.840000,11.290000,2.830233,1.858407,0.290089,0.535714,1.042945,0.209059,0.656934,3.333333,...,0.922619,1.528662,2.401216,1.286449,2.169492,1.192661,0.600000,1.772429,1.000000,1051.000000
max,38.660000,38.130000,7.826000,9.677419,3.894009,11.333333,7.647059,4.571429,6.744186,13.636364,...,3.809524,4.693878,4.320988,6.315789,7.096774,6.666667,5.454545,8.333333,1.000000,1111.000000
